In [34]:
import openai
from dotenv import load_dotenv
import os
from langchain.vectorstores import Chroma
import gradio as gr
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [35]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [36]:
vector_store = Chroma(
    embedding_function=embedding_function,
    persist_directory="../chroma",
    collection_name="osagyefo_v1",
)

In [37]:
retriever = vector_store.as_retriever(
    search_type="mmr",  # Maximal Marginal Relevance
    search_kwargs={"k": 5, "lambda_mult": 0.5}
)

In [38]:
print(vector_store._collection.count())  

1283


In [39]:
retriever.get_relevant_documents("my mother did not give me food")

[Document(metadata={'chapter_title': 'CHAPTER EIGHT', 'file_name': 'criminal offences act.pdf', 'doc_type': 'act', 'total_pages': 115}, page_content='CHAPTER EIGHT\nPublic Nuisance\nHindering Burials\n285.   Hindering burial of dead body\nA person who unlawfully hinders the burial of the d ead body of a person, or without lawful authority\ndisinters, dissects or harms the dead body of a person, or being under a duty to cause the dead body of a\nperson to be buried, fails to perform that duty commits a misdemeanour.\nUnwholesome Food\n286.   Selling unwholesome food\nA person who sells, or prepares or offers for sale, as being fit for consumption as food or drink, a thing\nwhich that person knows or has reason to believe that it is in a condition of putrefaction, adulteration, or\nother cause, as to be likely to be noxious to health, commits a misdemeanour.\nNoxious Trade\n287.   Carrying on of noxious trade, interference with public rights\nA person who, without lawful authority or exc

In [40]:
load_dotenv(override=True)
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

model_1 = "gemma2-9b-it"
client1 = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY"),
)

model_2 = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
client2 = openai.OpenAI(
  api_key=os.environ.get("TOGETHER_API_KEY"),
  base_url="https://api.together.xyz/v1",
)

PIPELINE

In [41]:
"""
LLM 1 will rephrase every question into 3 different similar questions--gemma2-9b-it

"""

def rephrase_query(user_query):
    pdf_path = "../docs"
    prompt_path = "../prompts/rephaser_llm_template.txt"
    
    with open(prompt_path, "r", encoding="utf-8") as f:
        system_prompt = f.read().strip()
        system_prompt += "Your knowledge is limited strictly to the content of the following files:"
        for filename in os.listdir(pdf_path):
            system_prompt += filename + "\n"


    message = [{'role': "system", "content": system_prompt}, {'role': "system", "content": user_query}]
    rephrased_query = client1.chat.completions.create(model=model_1, messages=message)
    rephrased_query = rephrased_query.choices[0].message.content

    return rephrased_query

In [42]:
# test
rephrased_query = rephrase_query("i mistakenly hit someone with my car")
print(rephrased_query)

context = "" 
relevant_docs = retriever.get_relevant_documents(rephrased_query)

for doc in relevant_docs:
    context += f"[Source: {doc.metadata['file_name']}, Chapter: {doc.metadata.get('chapter_title', 'N/A')}]\n{doc.page_content}\n\n"

['civil liability for vehicular accident', 'legal consequences of striking person with vehicle', 'liability under Ghanaian law for causing harm with motor vehicle'] 



In [43]:
context

'[Source: ghana_constitution.pdf, Chapter: Chapter IV]\nChapter IV\n-The Laws of Ghana\n\n[Source: criminal offences act.pdf, Chapter: CHAPTER EIGHT]\n(s)        in a town wilfully or negligently causes or pe rmits a vehicle, or a thing carried by the\nvehicle, or cattle, to damage a drain, ditch, or trench, at the side of a street, or a bridge, or a\npart of it respectively; or\n            (t)        in a town, without the written consent of the local authority or the Ghana Highway\nAuthority, wilfully displaces or takes up or damages the pavement, stones, or material of a\npublic way, or attempts to change or obstruct an outer course; or91(91)\n            (u)      assembles with other persons in a public place, or in an open space near a public place, for an\nidle, a vicious, or a disorderly purpose, or ot herwise than in the regular performance, or in\npursuance, of a lawful calling or object, to the annoyance or  obstruction of a passenger or\nperson frequenting the public place,

In [44]:
def chat_osagyefo(user_query, history):
    
    prompt_path = "../prompts/main_prompt_template.txt"

    rephrased_query = rephrase_query(user_query)
    relevant_docs = retriever.get_relevant_documents(rephrased_query)

    print("RELEVANT DOCS: ", relevant_docs)
    context = ''
    for doc in relevant_docs:
        context += f"[Source: {doc.metadata['file_name']}, Chapter: {doc.metadata.get('chapter_title', 'N/A')}]\n{doc.page_content}\n\n"

    # loading the prompt
    with open(prompt_path, "r", encoding="utf-8") as f:
        osagyefo_system_prompt = f.read().strip()
    osagyefo_system_prompt += f"Use the following context to answer:\n{context}"


    history = [
        {'role': h['role'], 'content': h['content'] }
        for h in history
        if 'role' in h and 'content' in h
    ]

    msg_to_model = [{"role": "system", "content": osagyefo_system_prompt}] + history + [{"role": "user", "content": user_query}]

    model = client2.chat.completions.create(
        model=model_2,
        messages=msg_to_model,
    )

    osagyefo_response = model.choices[0].message.content
    

    history.append({"role": "user", "content": user_query})
    history.append({"role": "assistant", "content": osagyefo_response})

    print('history', history)
    
    return osagyefo_response

In [45]:
demo_chatbot = gr.ChatInterface(chat_osagyefo,
                                title="OSAGYEFO ESQ.",
                                description="Welcome! Put forth your legal challenges. ")

c:\Users\DeLL\Desktop\anything_py\osagyefo_ai\.venv\lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [ ]:
demo_chatbot.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


RELEVANT DOCS:  [Document(metadata={'file_name': 'criminal offences act.pdf', 'chapter_title': 'CHAPTER TWO', 'doc_type': 'act', 'total_pages': 115}, page_content='CHAPTER TWO\nMurder and Similar Offences\nMurder and Manslaughter\n46. Murder.\n47. Definition of murder.\n48. Attempt to commit murder.\n49. Attempt to commit murder by convict.\n49A. Genocide.\n50. Manslaughter.\n51. Definition of manslaughter.\n52. Intentional murder reduced to manslaughter.\n53. Provocation.\n54. Exclusion of benefit of provocation.\n55. Mistake as to matter of provocation.\n56. Mistake as to person giving provocation.\nSuicide and Abortion\n57. Abetment of suicide.\n58. Abortion or miscarriage.\n59. Explanation as to causing abortion.\nCausing Harm to Child at Birth and Concealment of Birth\n60. Causing harm to child at birth.\n61. Explanation as to causing harm to child at birth.\n62. Concealment of body of child.\n63. Explanation as to concealment of body of child.\nSpecial Provisions relating to Murd